# Домашнее задание № 3. Исправление опечаток

## 1. Учет грамматики при оценке исправлений (3 балла)

В последнюю итерацию алгоритма для генерации исправлений добавьте еще один компонент - учет грамматической информации. Частично она уже учитывается за счет языковой модели (вероятность предсказывается для словоформы), но такой подход ограничен из-за того, что модель не может ничего предсказать для словоформ, которых не было в обучающей выборке. Чтобы это исправить постройте еще одну "языковую модель" на грамматических тэгах:
1) Используя mystem или pymorphy, разметьте какой-нибудь корпус (например, кусок wiki из семинара) или воспользуйтесь уже размеченным корпусом (например, opencorpora)
2) соберите униграмные и биграмные статистики на уровне грамматических тэгов (например, вместо `задача важна` у вас будет биграм `S,жен,неод=им,ед A=ед,кр,жен`). Для простоты можете начать только с частеречных тэгов и добавить остальную информацию позже
3) напишите функцию, которая будет оценивать вероятность данного предложения на основе грамматической языковой модели (статистик из предыдущего шага). Функция должна сначала преобразовать текст в грамматические тэги, используя точно такой же подход, что использовался на шаге 1.
4) в функции correct_text_with_lm замените compute_sentence_proba на вашу новую функцию и прогоните получившийся алгоритм на данных
5) сравните предсказания с предсказанием изначального correct_text_with_lm, проверьте метрики и посмотрите на различие в ошибках и исправлениях, найдите несколько примеров отличий в предсказаниях этих подходов

In [ ]:
!pip install --break-system-packages pymystem3

!pip install textdistance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2
!unzip '/content/drive/MyDrive/Colab Notebooks/data.zip'

--2025-12-05 18:53:13--  https://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2
Resolving opencorpora.org (opencorpora.org)... 104.21.15.199, 172.67.163.210, 2606:4700:3031::ac43:a3d2, ...
Connecting to opencorpora.org (opencorpora.org)|104.21.15.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1435452 (1.4M) [application/x-bzip2]
Saving to: ‘annot.opcorpora.no_ambig_strict.xml.bz2’

annot.opcorpora.no_ 100%[===================>]   1.37M  1.21MB/s    in 1.1s    

2025-12-05 18:53:14 (1.21 MB/s) - ‘annot.opcorpora.no_ambig_strict.xml.bz2’ saved [1435452/1435452]

Archive:  /content/drive/MyDrive/Colab Notebooks/data.zip
  inflating: correct_sents.txt       
  inflating: sents_with_mistakes.txt  
  inflating: wiki_data.txt           
  inflating: __MACOSX/._wiki_data.txt  


In [ ]:
import bz2
from lxml import etree
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from collections import Counter
from string import punctuation
punctuation += "«»—…“”•"
punct = set(punctuation)
from pymystem3 import Mystem
mystem = Mystem()
import textdistance
import re
from tqdm.notebook import tqdm

In [ ]:
bad = open('/content/sents_with_mistakes.txt', encoding='utf-8').read().splitlines()
true = open('/content/correct_sents.txt', encoding='utf-8').read().splitlines()
corpus = open('/content/wiki_data.txt', encoding='utf8').read()

In [ ]:
with bz2.open('annot.opcorpora.no_ambig_strict.xml.bz2', 'rb') as f_in, open('annot.opcorpora.no_ambig_strict.xml', 'wb') as f_out:
    f_out.write(f_in.read())

In [ ]:
open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())
vocab = Counter(re.findall('\w+', corpus.lower()))

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-2346103062.py:2: SyntaxWarning: invalid escape sequence '\w'
  vocab = Counter(re.findall('\w+', corpus.lower()))


In [ ]:
def multi_tokenizer(corpora):
    tokens_inf = []
    for sentence in corpora.xpath('//tokens'):
        for token in sentence.xpath('token'):
            word = token.xpath('@text')
            if word[0] not in punct:
                gram_info = token.xpath('tfr/v/l/g/@v')
                tokens_inf.append(gram_info)
    return tokens_inf

In [ ]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [ ]:
def unigram_stat(tokens):
  morpho_dict = Counter()
  for token in tokens:
      morpho_dict[tuple(token)] += 1
  return morpho_dict

In [ ]:
def bigram_stat(tokens):
    morpho_dict = Counter()
    for i in range(len(tokens) - 1):
        morpho_dict[(*tokens[i], *tokens[i+1])] += 1
    return morpho_dict

In [ ]:
def normalize(text):
    normalized_text = [word.strip(punctuation) for word in text.split()]
    normalized_text = [word.lower() for word in normalized_text if word]
    return normalized_text

In [ ]:
corp_inf = multi_tokenizer(open_corpora)
gram_unigrams = unigram_stat(corp_inf)
gram_bigrams = bigram_stat(corp_inf)

In [ ]:
words = normalize(corpus)
word_unigrams = Counter(words)
word_bigrams = Counter(ngrammer(words))

In [ ]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [ ]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])

    # вся эффективноть берется из того, что мы сразу считаем близость
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее

    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

In [ ]:
def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()

    for word in lookup:
        similarities[word] = metric.similarity(text, word)

    return similarities.most_common(topn)

In [ ]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)


    return closest

In [ ]:
def mystem_gr_to_tags(gr):
    return tuple(gr.replace('=', ',').split(','))

In [ ]:
def compute_sentence_proba(text, word_counts=gram_unigrams, bigram_counts=gram_bigrams):
    prob = 0
    analyzed = mystem.analyze(text)
    tags_seq = []
    for word in analyzed:
        w = word.get('text', '').strip()
        if w and w not in punct:
            if 'analysis' in word and word['analysis']:
                gr_str = word['analysis'][0]['gr']
                tags = mystem_gr_to_tags(gr_str)
                tags_seq.append(tags)
            else:
                # Слово не разобрано — добавляем UNK
                tags_seq.append(('UNK',))


    seq = [('<start>',)] + tags_seq + [('<end>',)]


    for i in range(len(seq) - 1):
        word1 = seq[i]
        word2 = seq[i + 1]
        bigram_key = word1 + word2

        if word1 in word_counts and bigram_key in bigram_counts:
            prob += np.log(bigram_counts[bigram_key] / word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob

In [ ]:
def compute_sentence_proba_prev(text, word_counts=word_unigrams, bigram_counts=word_bigrams):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob

In [ ]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [ ]:
def correct_text_with_lm(text, func):
    sentence = normalize(text)
    corrections = []

    for word in sentence:
        if predict_mistaken(word, vocab):
            preds = get_closest_hybrid_match(word, X, vec)
            preds = [p[0] for p in preds]
            corrections.append(preds)
        else:
            corrections.append([word])

    possible_sentences = [" ".join(words) for words in itertools.product(*corrections)]
    most_prob_sentence = max(possible_sentences, key=lambda x: func(x))

    return most_prob_sentence

In [ ]:
for i in range(20):
    cor_gram = correct_text_with_lm(bad[i], compute_sentence_proba)
    cor_class = correct_text_with_lm(bad[i], compute_sentence_proba_prev)

    if cor_gram != cor_class:
        print(bad[i])
        print('Грамматический:')
        print(cor_gram)
        print('Классический:')
        print(cor_class)
        print()

Основая цель мероприятия - практическая отработка навыков по оказанию помощи гражданам, попавшим в ДТП, а также повышение и совершенствование уровня профессиональной подготовки сотрудников МЧС при проведении аварийно-спасательных работ по ликвидации последствий дорожно-транспортных происшествий, сокращение временных показателей реагирования.
Грамматический:
сосновая цель мероприятия практическая отработка навыков по оказанию помощи гражданам попавшим в дтп а также повышение и совершенствование уровня профессиональной подготовки сотрудников мчс при проведении горноспасательных работ по ликвидации последствий автотранспортных происшествий сокращение временных показателей реагирования
Классический:
сосновая цель мероприятия практическая отработка навыков по оказанию помощи гражданам попавшим в дтп а также повышение и совершенствование уровня профессиональной подготовки сотрудников мчс при проведении спасательных работ по ликвидации последствий транспортных происшествий сокращение временны

In [ ]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

In [ ]:
y_true = []
y_actual = []
y_preds = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    corrected = correct_text_with_lm(bad[i], compute_sentence_proba).split()

    for i in range(len(word_pairs)):
        true_word = word_pairs[i][0]
        actual_word = word_pairs[i][1]
        pred = corrected[i]

        y_preds.append(pred)
        y_true.append(true_word)
        y_actual.append(actual_word)

In [ ]:
def calculate_metrics(true, actual, predicted):
    correct = 0
    total = 0

    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    for i in range(len(true)):
        t, a, p = true[i], actual[i], predicted[i]

        # общая точность
        if t == p:
            correct += 1
        total += 1

        # % сломанных правильных слов
        if t == a:
            total_correct += 1
            if t != p:
                correct_broken += 1
        # % процент правильно исправленных настоящих ошибок
        else:
            total_mistaken += 1
            if t == p:
                mistaken_fixed += 1

    return {"total_accuracy": correct/total,
            "fixed_mistakes": mistaken_fixed/total_mistaken,
            "broken_correct_words": correct_broken/total_correct}

In [ ]:
metrics = calculate_metrics(y_true, y_actual, y_preds)
metrics

{'total_accuracy': 0.8480649784997611,
 'fixed_mistakes': 0.4267399267399267,
 'broken_correct_words': 0.08873626373626374}

'total_accuracy': 0.847223611805903,

 'fixed_mistakes': 0.4231366459627329,

 'broken_correct_words': 0.09004249454461927

In [ ]:
y_true_1 = []
y_actual_1 = []
y_preds_1 = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    corrected = correct_text_with_lm(bad[i], compute_sentence_proba_prev).split()

    for i in range(len(word_pairs)):
        true_word = word_pairs[i][0]
        actual_word = word_pairs[i][1]
        pred = corrected[i]

        y_preds_1.append(pred)
        y_true_1.append(true_word)
        y_actual_1.append(actual_word)

In [ ]:
metrics = calculate_metrics(y_true_1, y_actual_1, y_preds_1)
metrics

'total_accuracy': 0.8477238619309655,

 'fixed_mistakes': 0.42701863354037267,

 'broken_correct_words': 0.09004249454461927

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (обратите внимание, что для одного удаления может быть несколько правильных слов!)
3) При исправлении слова с опечаткой сначала само слово проверятся по словарю удаления, а затем для этого слова генерируются все варианты удалений, и каждый вариант проверяется по словарю удалений. Если в словаре удалений таким образом находится совпадение, то соответствующее ему правильное слово становится исправлением.
Если совпадений несколько, то выбирается наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [ ]:
from collections import defaultdict

In [ ]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

In [ ]:
delete_dict = defaultdict(list)

for word in vocab:
    if len(word) >= 2:
        for i in range(len(word)):
            deletion = word[:i] + word[i+1:]
            delete_dict[deletion].append(word)

In [ ]:
def correction(word):

    if word in vocab:
        return word

    candidates = set()

    if word in delete_dict:
        candidates.update(delete_dict[word])

    for i in range(len(word)):
        deletion = word[:i] + word[i+1:]
        if deletion in delete_dict:
            candidates.update(delete_dict[deletion])

    if candidates:
        return max(candidates, key=lambda w: vocab.get(w, 0))

    return word

In [ ]:
cashed = {}
y_true_2 = []
y_actual_2 = []
y_preds_2 = []
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])

    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше

        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        y_preds_2.append(predicted)
        y_true_2.append(pair[0])
        y_actual_2.append(pair[1])

  0%|          | 0/915 [00:00<?, ?it/s]

In [ ]:
metrics = calculate_metrics(y_true_2, y_actual_2, y_preds_2)

In [ ]:
metrics

{'total_accuracy': 0.8681340670335168,
 'fixed_mistakes': 0.30745341614906835,
 'broken_correct_words': 0.04892615137245894}

'total_accuracy': 0.8681340670335168,

 'fixed_mistakes': 0.30745341614906835,

 'broken_correct_words': 0.04892615137245894

# Задание 3 (2 балла)

Используя любой из алгоритмов из семинара или домашки, детально проанализируйте получаемые ошибки. Улучшите алгоритм так, чтобы исправить ошибки. Улучшения в алгоритме должны быть общими, не привязанными к конкретным словам (например, словарь исключений не будет считаться). За каждое улучшение, которое исправляет 5+ ошибок вы получите 0.5 балла (максимум 2 в целом)

In [ ]:
def count_errors_symspell(bad_sents, true_sents, cor_func):

    total_errors = 0
    total_words = 0
    cache = {}

    for i in range(len(true_sents)):
        word_pairs = align_words(true_sents[i], bad_sents[i])
        for true_word, mistaken_word in word_pairs:
            if mistaken_word in cache:
                pred = cache[mistaken_word]
            else:
                pred = cor_func(mistaken_word)
                cache[mistaken_word] = pred

            total_words += 1
            if true_word != pred:
                total_errors += 1
                print(f"{true_word} ← {mistaken_word} → {pred}")

    accuracy = (total_words - total_errors) / total_words if total_words > 0 else 0

    print(f"{total_errors} ошибок из {total_words} слов")
    print(f"Точность: {accuracy:.3f}")


In [ ]:
count_errors_symspell(bad, true, correction)

симпатичнейшее ← симпатичнейшое → симпатичнейшое
шпионское ← шпионское → шпионской
гламурный ← гламурный → гламурным
clap ← clap → camp
поясним ← пояним → эпоним
язычки ← язычки → язычок
очень ← оччччень → оччччень
насчет ← нащщот → нащщот
сетуют ← сетуют → сетует
основная ← основая → основан
в ← вобщем → вообще
общем ← как → как
как ← вы → вы
вы ← знаете → знаете
знаете ← из → из
из ← моего → моего
моего ← не → не
недавнего ← давнего → давнего
ящика ← ящека → щенка
рите ← рите → риме
снятся ← снятся → снялся
потому ← патаму → панаму
что ← шта → шта
пытаюсь ← пытаюсь → пытаясь
дубраве ← дубраве → дубравы
повтыкав ← поффтыкав → поффтыкав
билетным ← билетным → билетные
ссоре ← соре → соре
что-то ← чтото → торто
кредиток ← кредиток → кредитов
кормилицу ← кормилицу → кормилица
молодежь ← молодеж → молодеж
участвовать ← учавствовать → учавствовать
сегодня ← седня → седан
навешать ← вешать → вешать
это ← эт → эт
конечно ← канешна → казнена
4:0 ← 4:0 → 400
начальник ← начальнег → начальное
за

1318 ошибок из 9995 слов

Точность: 0.868

In [ ]:
def correction_1(word):

    if not word.isalpha():
        return word
    if any(c.isupper() for c in word[1:]):
        return word

    if word in vocab:
        return word

    candidates = set()

    if word in delete_dict:
        candidates.update(delete_dict[word])

    for i in range(len(word)):
        deletion = word[:i] + word[i+1:]
        if deletion in delete_dict:
            candidates.update(delete_dict[deletion])

    if candidates:
        return max(candidates, key=lambda w: vocab.get(w, 0))

    return word

In [ ]:
count_errors_symspell(bad, true, correction_1)

симпатичнейшее ← симпатичнейшое → симпатичнейшое
шпионское ← шпионское → шпионской
гламурный ← гламурный → гламурным
clap ← clap → camp
поясним ← пояним → эпоним
язычки ← язычки → язычок
очень ← оччччень → оччччень
насчет ← нащщот → нащщот
сетуют ← сетуют → сетует
основная ← основая → основан
в ← вобщем → вообще
общем ← как → как
как ← вы → вы
вы ← знаете → знаете
знаете ← из → из
из ← моего → моего
моего ← не → не
недавнего ← давнего → давнего
ящика ← ящека → щенка
рите ← рите → риме
снятся ← снятся → снялся
потому ← патаму → панаму
что ← шта → шта
пытаюсь ← пытаюсь → пытаясь
дубраве ← дубраве → дубравы
повтыкав ← поффтыкав → поффтыкав
билетным ← билетным → билетные
ссоре ← соре → соре
что-то ← чтото → торто
кредиток ← кредиток → кредитов
кормилицу ← кормилицу → кормилица
молодежь ← молодеж → молодеж
участвовать ← учавствовать → учавствовать
сегодня ← седня → седан
навешать ← вешать → вешать
это ← эт → эт
конечно ← канешна → казнена
начальник ← начальнег → начальное
зажег ← зажог → за

1305 ошибок из 9995 слов

Точность: 0.869

In [ ]:
def correction_2(word):

    if not word.isalpha():  # содержит цифры, точки, дефисы и т.д.
        return word
    if any(c.isupper() for c in word[1:]):  # CamelCase или случайные заглавные (редко в середине ошибок)
        return word

    if word in vocab:
        return word

    candidates = set()

    if word in delete_dict:
        candidates.update(delete_dict[word])

    for i in range(len(word)):
        deletion = word[:i] + word[i+1:]
        if deletion in delete_dict:
            candidates.update(delete_dict[deletion])


    MIN_FREQ = 13
    candidates = {w for w in candidates if vocab.get(w, 0) >= MIN_FREQ}

    if candidates:
        return max(candidates, key=lambda w: vocab.get(w, 0))

    return word

In [ ]:
count_errors_symspell(bad, true, correction_2)

симпатичнейшее ← симпатичнейшое → симпатичнейшое
апофеозом ← опофеозом → опофеозом
поясним ← пояним → пояним
получатся ← полчатся → полчатся
очень ← оччччень → оччччень
насчет ← нащщот → нащщот
основная ← основая → основан
напрасно ← нарасно → нарасно
в ← вобщем → вообще
общем ← как → как
как ← вы → вы
вы ← знаете → знаете
знаете ← из → из
из ← моего → моего
моего ← не → не
недавнего ← давнего → давнего
ящика ← ящека → ящека
предлагаю ← предлагю → предлагю
сегодняшнее ← сегодяшнее → сегодяшнее
рите ← рите → риме
снятся ← снятся → снялся
потому ← патаму → патаму
что ← шта → шта
пытаюсь ← пытаюсь → пытаясь
повтыкав ← поффтыкав → поффтыкав
ссоре ← соре → соре
что-то ← чтото → чтото
молодежь ← молодеж → молодеж
участвовать ← учавствовать → учавствовать
сегодня ← седня → седня
навешать ← вешать → вешать
это ← эт → эт
конечно ← канешна → канешна
начальник ← начальнег → начальное
зажег ← зажог → залог
по-взрослому ← павзрослому → павзрослому
предыдущую ← предудущую → предудущую
слег ← слег → 

1145 ошибок из 9995 слов

Точность: 0.885

In [ ]:
def correction_3(word):
    if word in vocab:
        return word

    candidates = set()

    if word in delete_dict:
        candidates.update(delete_dict[word])

    for i in range(len(word)):
        deletion = word[:i] + word[i+1:]
        if deletion in delete_dict:
            candidates.update(delete_dict[deletion])

    MIN_FREQ = 13
    candidates = {w for w in candidates if vocab.get(w, 0) >= MIN_FREQ}


    MAX_DISTANCE = 1
    candidates = {w for w in candidates if textdistance.levenshtein.distance(word, w) <= MAX_DISTANCE}

    if candidates:
        return max(candidates, key=lambda w: vocab.get(w, 0))

    return word

In [ ]:
count_errors_symspell(bad, true, correction_3)

симпатичнейшее ← симпатичнейшое → симпатичнейшое
апофеозом ← опофеозом → опофеозом
поясним ← пояним → пояним
получатся ← полчатся → полчатся
очень ← оччччень → оччччень
насчет ← нащщот → нащщот
основная ← основая → основан
напрасно ← нарасно → нарасно
в ← вобщем → вобщем
общем ← как → как
как ← вы → вы
вы ← знаете → знаете
знаете ← из → из
из ← моего → моего
моего ← не → не
недавнего ← давнего → давнего
ящика ← ящека → ящека
предлагаю ← предлагю → предлагю
сегодняшнее ← сегодяшнее → сегодяшнее
рите ← рите → риме
снятся ← снятся → снялся
потому ← патаму → патаму
что ← шта → шта
пытаюсь ← пытаюсь → пытаясь
повтыкав ← поффтыкав → поффтыкав
ссоре ← соре → соре
что-то ← чтото → чтото
молодежь ← молодеж → молодеж
участвовать ← учавствовать → учавствовать
сегодня ← седня → седня
навешать ← вешать → вешать
это ← эт → эт
конечно ← канешна → канешна
4:0 ← 4:0 → 400
начальник ← начальнег → начальнег
зажег ← зажог → залог
по-взрослому ← павзрослому → павзрослому
предыдущую ← предудущую → предудущу

1101 ошибок из 9995 слов

Точность: 0.890

In [ ]:
def correction_4(word):

    if word in vocab:
        return word


    candidates = set()

    if word in delete_dict:
        candidates.update(delete_dict[word])

    if len(word) > 4:

        for i in range(len(word)):
            deletion = word[:i] + word[i+1:]
            if deletion in delete_dict:
                candidates.update(delete_dict[deletion])

    MIN_FREQ = 13
    candidates = {w for w in candidates if vocab.get(w, 0) >= MIN_FREQ}

    MAX_DISTANCE = 1
    candidates = {w for w in candidates if textdistance.levenshtein.distance(word, w) <= MAX_DISTANCE}

    if candidates:
        return max(candidates, key=lambda w: vocab.get(w, 0))

    return word

In [ ]:
count_errors_symspell(bad, true, correction_4)

симпатичнейшее ← симпатичнейшое → симпатичнейшое
апофеозом ← опофеозом → опофеозом
поясним ← пояним → пояним
получатся ← полчатся → полчатся
очень ← оччччень → оччччень
насчет ← нащщот → нащщот
основная ← основая → основан
напрасно ← нарасно → нарасно
в ← вобщем → вобщем
общем ← как → как
как ← вы → вы
вы ← знаете → знаете
знаете ← из → из
из ← моего → моего
моего ← не → не
недавнего ← давнего → давнего
ящика ← ящека → ящека
предлагаю ← предлагю → предлагю
сегодняшнее ← сегодяшнее → сегодяшнее
снятся ← снятся → снялся
потому ← патаму → патаму
что ← шта → шта
пытаюсь ← пытаюсь → пытаясь
повтыкав ← поффтыкав → поффтыкав
ссоре ← соре → соре
что-то ← чтото → чтото
молодежь ← молодеж → молодеж
участвовать ← учавствовать → учавствовать
сегодня ← седня → седня
навешать ← вешать → вешать
это ← эт → эт
конечно ← канешна → канешна
начальник ← начальнег → начальнег
зажег ← зажог → залог
по-взрослому ← павзрослому → павзрослому
предыдущую ← предудущую → предудущую
подсаживается ← подсаживаеться → 

1089 ошибок из 9995 слов

Точность: 0.891